# การสร้างด้วยโมเดล Mistral

## บทนำ

บทเรียนนี้จะครอบคลุม:
- การสำรวจโมเดล Mistral ที่แตกต่างกัน
- การเข้าใจกรณีการใช้งานและสถานการณ์สำหรับแต่ละโมเดล
- ตัวอย่างโค้ดที่แสดงคุณสมบัติเฉพาะของแต่ละโมเดล


## โมเดล Mistral

ในบทเรียนนี้ เราจะสำรวจโมเดล Mistral 3 แบบที่แตกต่างกัน:  
**Mistral Large**, **Mistral Small** และ **Mistral Nemo**

โมเดลเหล่านี้แต่ละตัวมีให้ใช้ฟรีบนตลาดโมเดล Github โค้ดในโน้ตบุ๊กนี้จะใช้โมเดลเหล่านี้ในการรันโค้ด นี่คือรายละเอียดเพิ่มเติมเกี่ยวกับการใช้โมเดล Github เพื่อ [สร้างต้นแบบด้วยโมเดล AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst)


## Mistral Large 2 (2407)
Mistral Large 2 เป็นโมเดลเรือธงปัจจุบันจาก Mistral และออกแบบมาเพื่อการใช้งานในองค์กร

โมเดลนี้เป็นการอัปเกรดจาก Mistral Large ดั้งเดิมโดยมี
- หน้าต่างบริบทที่ใหญ่ขึ้น - 128k เทียบกับ 32k
- ประสิทธิภาพที่ดีขึ้นในงานคณิตศาสตร์และการเขียนโค้ด - ความแม่นยำเฉลี่ย 76.9% เทียบกับ 60.4%
- ประสิทธิภาพหลายภาษาเพิ่มขึ้น - ภาษาที่รองรับได้แก่: อังกฤษ, ฝรั่งเศส, เยอรมัน, สเปน, อิตาลี, โปรตุเกส, ดัตช์, รัสเซีย, จีน, ญี่ปุ่น, เกาหลี, อาหรับ และ ฮินดี

ด้วยคุณสมบัติเหล่านี้ Mistral Large โดดเด่นในด้าน
- *Retrieval Augmented Generation (RAG)* - เนื่องจากหน้าต่างบริบทที่ใหญ่ขึ้น
- *Function Calling* - โมเดลนี้มีการเรียกฟังก์ชันในตัวซึ่งช่วยให้สามารถรวมกับเครื่องมือและ API ภายนอกได้ การเรียกเหล่านี้สามารถทำได้ทั้งแบบขนานหรือทีละคำสั่งตามลำดับ
- *Code Generation* - โมเดลนี้โดดเด่นในการสร้างโค้ด Python, Java, TypeScript และ C++


### ตัวอย่าง RAG โดยใช้ Mistral Large 2


ในตัวอย่างนี้ เราใช้ Mistral Large 2 เพื่อรันรูปแบบ RAG บนเอกสารข้อความ คำถามถูกเขียนเป็นภาษาเกาหลีและถามเกี่ยวกับกิจกรรมของผู้เขียนก่อนเข้ามหาวิทยาลัย

มันใช้ Cohere Embeddings Model เพื่อสร้าง embeddings ของเอกสารข้อความรวมถึงคำถาม สำหรับตัวอย่างนี้ ใช้แพ็กเกจ Python faiss เป็นที่เก็บเวกเตอร์

พรอมต์ที่ส่งไปยังโมเดล Mistral รวมทั้งคำถามและชิ้นส่วนที่ดึงมาได้ซึ่งมีความคล้ายคลึงกับคำถาม จากนั้นโมเดลจะให้คำตอบในรูปแบบภาษาธรรมชาติ


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small เป็นโมเดลอีกตัวหนึ่งในตระกูล Mistral ภายใต้หมวดหมู่ premier/enterprise ตามชื่อที่บ่งบอก โมเดลนี้เป็น Small Language Model (SLM) ข้อดีของการใช้ Mistral Small คือ: 
- ประหยัดค่าใช้จ่ายเมื่อเทียบกับ Mistral LLMs เช่น Mistral Large และ NeMo - ลดราคาลง 80%
- ความหน่วงต่ำ - ตอบสนองได้เร็วกว่า LLMs ของ Mistral
- ยืดหยุ่น - สามารถปรับใช้ในสภาพแวดล้อมต่าง ๆ ได้โดยมีข้อจำกัดน้อยลงในเรื่องทรัพยากรที่ต้องการ

Mistral Small เหมาะสำหรับ: 
- งานที่ใช้ข้อความเป็นหลัก เช่น การสรุปความ การวิเคราะห์อารมณ์ และการแปลภาษา
- แอปพลิเคชันที่มีการร้องขอบ่อยครั้งเนื่องจากความคุ้มค่าเรื่องต้นทุน
- งานโค้ดที่ต้องการความหน่วงต่ำ เช่น การตรวจสอบและแนะนำโค้ด


## การเปรียบเทียบ Mistral Small และ Mistral Large

เพื่อแสดงความแตกต่างของความหน่วงระหว่าง Mistral Small และ Large ให้รันเซลล์ด้านล่างนี้

คุณควรเห็นความแตกต่างของเวลาตอบสนองระหว่าง 3-5 วินาที รวมถึงความยาวและสไตล์ของการตอบสนองในคำสั่งเดียวกันด้วยเช่นกัน


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

เมื่อเทียบกับโมเดลอีกสองตัวที่กล่าวถึงในบทเรียนนี้ Mistral NeMo เป็นโมเดลฟรีเพียงตัวเดียวที่มีใบอนุญาต Apache2

โมเดลนี้ถูกมองว่าเป็นการอัปเกรดจาก LLM โอเพนซอร์สก่อนหน้าของ Mistral คือ Mistral 7B

คุณสมบัติอื่น ๆ ของโมเดล NeMo ได้แก่:

- *การแยกโทเค็นที่มีประสิทธิภาพมากขึ้น:* โมเดลนี้ใช้ตัวแยกโทเค็น Tekken แทนที่จะใช้ tiktoken ที่ใช้กันทั่วไป ซึ่งช่วยให้มีประสิทธิภาพที่ดีกว่าในหลายภาษาและโค้ด

- *การปรับแต่งเพิ่มเติม:* โมเดลฐานพร้อมสำหรับการปรับแต่งเพิ่มเติม ซึ่งช่วยให้มีความยืดหยุ่นมากขึ้นสำหรับกรณีการใช้งานที่อาจต้องการการปรับแต่ง

- *การเรียกฟังก์ชันแบบเนทีฟ* - เช่นเดียวกับ Mistral Large โมเดลนี้ได้รับการฝึกฝนให้รองรับการเรียกฟังก์ชัน ซึ่งทำให้เป็นหนึ่งในโมเดลโอเพนซอร์สแรก ๆ ที่ทำได้เช่นนี้


## Mistral NeMo

เมื่อเทียบกับโมเดลอีกสองตัวที่กล่าวถึงในบทเรียนนี้ Mistral NeMo เป็นโมเดลฟรีเพียงตัวเดียวที่มีใบอนุญาต Apache2

โมเดลนี้ถูกมองว่าเป็นการอัปเกรดจาก LLM โอเพนซอร์สรุ่นก่อนหน้าจาก Mistral คือ Mistral 7B

คุณสมบัติอื่น ๆ ของโมเดล NeMo ได้แก่:

- *การแยกโทเค็นที่มีประสิทธิภาพมากขึ้น:* โมเดลนี้ใช้ตัวแยกโทเค็น Tekken แทนที่จะใช้ tiktoken ที่ใช้กันทั่วไป ซึ่งช่วยให้มีประสิทธิภาพที่ดีกว่าในหลายภาษาและโค้ด

- *การปรับแต่งเพิ่มเติม:* โมเดลฐานพร้อมสำหรับการปรับแต่งเพิ่มเติม ซึ่งช่วยให้มีความยืดหยุ่นมากขึ้นสำหรับกรณีการใช้งานที่อาจต้องการการปรับแต่ง

- *การเรียกฟังก์ชันแบบเนทีฟ* - เช่นเดียวกับ Mistral Large โมเดลนี้ได้รับการฝึกฝนให้เรียกฟังก์ชันได้ ซึ่งทำให้เป็นหนึ่งในโมเดลโอเพนซอร์สแรก ๆ ที่ทำได้เช่นนี้


### การเปรียบเทียบ Tokenizers

ในตัวอย่างนี้ เราจะดูว่า Mistral NeMo จัดการการตัดคำอย่างไรเมื่อเทียบกับ Mistral Large

ตัวอย่างทั้งสองใช้ prompt เดียวกัน แต่คุณควรเห็นว่า NeMo คืน token กลับมาน้อยกว่าเมื่อเทียบกับ Mistral Large


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## การเรียนรู้ไม่ได้หยุดอยู่แค่นี้ ดำเนินการเดินทางต่อไป

หลังจากเรียนบทเรียนนี้เสร็จแล้ว ลองดู [คอลเลกชันการเรียนรู้ Generative AI ของเรา](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) เพื่อเพิ่มพูนความรู้ด้าน Generative AI ของคุณต่อไป!


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้เราจะพยายามให้ความถูกต้องสูงสุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใด ๆ ที่เกิดจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
